In [54]:
import scipy
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

In [93]:
class IterativeFitting:
    def __init__(self, L, A0, N, M1):
        self.N = N
        self.M1 = M1
        self.L = L
        self.A0 = A0

        self.n = L.shape[0]

    def convexProgram(self, constraints=[], N_const=True, M1_const=True):
        """Solves the convex minimization problem of the paper.
        Note that every "constraints" argument must have elements defined as lambda functions of the form:
            lambda A,N,M1: cp.sum(A) == 115 
        as an example. This is because A, N, M1 is not being defined until the function is being called.
        """
        print(len(constraints))
        if len(constraints) > 0:
            N_const = False
            M1_const = False
        if N_const:
            N = self.N
        else:
            N = cp.Variable(shape=(self.n + 1))
        if M1_const:
            M1 = self.M1
        else:
            M1 = cp.Variable()
        L = self.L
        A = cp.Variable(shape=self.n)
        constraints_eval = [c(A,N,M1) for c in constraints]
        obj = cp.Minimize(
            cp.scalar_product(-L, A) -
            cp.entr(M1 - cp.sum(A)) - (M1 - cp.sum(A)) +
            cp.sum(-cp.entr(N[1:] - A) - (N[1:] - A)) +
            cp.sum(-cp.entr(A) - A) -
            cp.entr(N[0] - M1 + cp.sum(A)) - (N[0] - M1 + cp.sum(A)) 
        )
        if len(constraints) > 0:
            problem = cp.Problem(obj, constraints_eval)
            problem.solve(solver=cp.ECOS)
            return A, N
        problem = cp.Problem(obj)
        problem.solve(solver=cp.ECOS)
        return A

    def GL_linesearch(self):
        """This implements our function with the line search over root-finding Newton's.
        Performs a line search.
        """
        N = self.N
        M1 = self.M1
        A = self.A0
        A1 = A
        diff = 1
        i = 1
        while diff > 1e-4:
            A1 = A
            Aplus = A.sum()
            a0 = self.M1 - Aplus
            b0 = self.N[0] - Aplus
            B = self.N[1:] - A
            c0 = 1/a0 + 1/b0
            c = 1/A + 1/B

            # Gradient step in Newton's Method and get ∆A
            e = self.L + np.log(a0) + np.log(B) - np.log(A) - np.log(b0)
            H = np.ones((self.n,self.n))*c0
            H += np.diag(c)
            dA = scipy.linalg.solve(H,e,assume_a="pos")
            dA_pos = np.where(dA > 0, dA, np.nan)
            dA_neg = np.where(dA < 0, dA, np.nan)

            # Taking minimum over positive change values
            A_pos = np.where(dA>0, A, np.nan)
            N_pos = np.where(dA>0, (N[1:]), np.nan)
            pre_alpha1 = (N_pos - A_pos)/(dA_pos)
            pre_alpha1 = np.nan_to_num(pre_alpha1, nan=1e10)
            alpha1 = np.min(pre_alpha1) # take smallest element from here

            # Taking minimum over negative change values
            A_neg = np.where(dA<0, A, np.nan)
            pre_alpha2 = -A_neg / dA_neg
            pre_alpha2 = np.nan_to_num(pre_alpha2, nan=1e10)
            alpha2 = np.min(pre_alpha2)

            # Choosing the smallest alpha (step size)
            alpha = 0.99*np.min(np.array([alpha1,alpha2,1]))
            # alpha = 0.5

            # Update A
            A += alpha*dA
            i += 1
            diff = np.linalg.norm(A1 - A)
        return A, a0, B, b0, i

    def GL(self):
        """Standard GL method.
        """
        N = self.N
        M1 = self.M1
        A = self.A0
        A1 = A
        diff = 1
        i = 1
        while diff > 1e-4:
            A1 = A
            Aplus = A.sum()
            a0 = self.M1 - Aplus
            b0 = self.N[0] - Aplus
            B = self.N[1:] - A
            c0 = 1/a0 + 1/b0
            c = 1/A + 1/B

            # Gradient step in Newton's Method and get ∆A
            e = self.L + np.log(a0) + np.log(B) - np.log(A) - np.log(b0)
            H = np.ones((self.n,self.n))*c0
            H += np.diag(c)
            A += scipy.linalg.solve(H,e,assume_a="pos")
            i += 1
            diff = np.linalg.norm(A1 - A)
        return A, a0, B, b0, i

In [94]:
x = np.array([2,6,11])
Nx = np.array([337,167,186,212])
M1x = 451
Lx = np.array([np.log(0.80),np.log(1.16),np.log(1.57)])
vx = np.array([0.0542,0.0563,0.0563])
A0 = M1x*Nx[1:]/(Nx.sum())

In [95]:
it_fit_ex = IterativeFitting(Lx,A0,Nx,M1x)

In [96]:
A = it_fit_ex.convexProgram()

0


In [97]:
A

Variable((3,), var1807)

In [98]:
A_GLN = it_fit_ex.GL_linesearch()

In [99]:
A_GLN

(array([ 79.93869991, 106.13845038, 136.85534803]),
 168.5,
 array([ 83.5,  93. , 106. ]),
 54.5,
 2)